## Assumptions

In the first notebook we have established the structure and contents of the `modelnet40v1` dataset.

Now, we will attempt to create a model that is capable of determining the type of the object based on all images of an object instance. The type of architecture that allows just that is called a *Multi-View Convolutional Network (`MVCNN`)* - you can read about it in more detail [in this arXiv paper](https://arxiv.org/pdf/1505.00880.pdf). In short: it allows feature extraction from multiple images, pools the individual image feature vectors using a symmetrical function and finally makes a prediction on the pooled vector. Because we are using a symmetrical function for pooling, the number of images per instance is arbitrary and can change from instance to instance.

![mvcnn_figure](./data/mvcnn_figure.JPG)

In our case, this experiment will be performed with the following configuration - we'll use:
* `ResNet50` pretrained on the `ImageNet` dataset as the backbone feature extractor for the individual images
* `mean` as the symmetrical pooling function for the individual vectors 
* `dropout` in the layers responsible for pooling and prediction
* `Adam` as the optimizer
* `F1-Macro` calculated on the validation set as the driving metric to reveal our champion model

In addition, because the experiment will be performed on a local machine with no access to a reasonable GPU, we will make some simplifications in the training pipeline:
* the number of classes in training will be limited to just 4: `airplane`, `bathtub`, `bed` and `bench`
* the pretrained feature extractor weights will remain frozen throughout the training
* at training time we'll use the batch dimension of the input to provide the instance images - instances will be forwarded through the net one by one, i.e. the *de facto* batch size will be 1

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

from dataset_classes.mvcnn_data_module import MVCNNDataModule
from model_classes.mvcnn import MVCNNClassifier
from model_classes.callbacks import UnfreezePretrainedWeights, ResetEvalResults

/mnt/d/coding/git/TomaszKaleczyc/MVCNN_training/environment/mvcnn_train/lib/python3.6/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


## Parameters

In [2]:
NUM_CLASSES = 4
LEARNING_RATE = 1e-3
LEARNING_RATE_REDUCTION_FACTOR = 1e3
NUM_EPOCHS = 20
NUM_EPOCHS_FREEZE_PRETRAINED = 20
BATCH_SIZE = 1
DROPOUT_RATE = 0.3
SAVE_PATH = './output'

## Class initialization

In [3]:
data_module = MVCNNDataModule(NUM_CLASSES, BATCH_SIZE)

Dataset type: TRAIN
------------------------------------------------------------
Class name: AIRPLANE
Total number of instances: 80
Total number of images: 960
------------------------------------------------------------
Class name: BATHTUB
Total number of instances: 80
Total number of images: 960
------------------------------------------------------------
Class name: BED
Total number of instances: 80
Total number of images: 960
------------------------------------------------------------
Class name: BENCH
Total number of instances: 80
Total number of images: 960
Dataset type: TEST
------------------------------------------------------------
Class name: AIRPLANE
Total number of instances: 20
Total number of images: 240
------------------------------------------------------------
Class name: BATHTUB
Total number of instances: 20
Total number of images: 240
------------------------------------------------------------
Class name: BED
Total number of instances: 20
Total number of images: 

In [4]:
model = MVCNNClassifier(
    learning_rate=LEARNING_RATE,
    num_epochs_freeze_pretrained=NUM_EPOCHS_FREEZE_PRETRAINED,
    dropout_rate=DROPOUT_RATE,
    )

Feature extractor weights frozen


In [5]:
callbacks = [
    ModelCheckpoint(monitor='val_f1', verbose=True, mode='max'),
    UnfreezePretrainedWeights(LEARNING_RATE_REDUCTION_FACTOR),
    ResetEvalResults(NUM_CLASSES)
]

In [6]:
trainer = Trainer(
    max_epochs=NUM_EPOCHS,
    fast_dev_run=False,
    default_root_dir=SAVE_PATH,
    callbacks=callbacks
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


## Model training

In [7]:
trainer.fit(
    model,
    train_dataloader=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader()
)


  | Name                 | Type             | Params
----------------------------------------------------------
0 | feature_extractor    | FeatureExtractor | 25.6 M
1 | image_vector_creator | Sequential       | 512 K 
2 | predictor            | Sequential       | 74.5 K
----------------------------------------------------------
586 K     Trainable params
25.6 M    Non-trainable params
26.1 M    Total params
104.576   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]


Eval stats:
TP per class: [0 0 0 0] Average: 0.00
TN per class: [0 2 2 0] Average: 1.00
FP per class: [0 0 0 2] Average: 0.50
FN per class: [2 0 0 0] Average: 0.50
ACCURACY per class: [0. 1. 1. 0.] Average: 0.50
PRECISION per class: [0. 0. 0. 0.] Average: 0.00
RECALL per class: [0. 0. 0. 0.] Average: 0.00
F1 per class: [0. 0. 0. 0.] Average: 0.00
Avg Loss val tensor(1.4337)
F1 val tensor(0.)
Accuracy VAL tensor(0.7500)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch 18, global step 6079: val_f1 was not in top 1



Eval stats:
TP per class: [13 14  9 16] Average: 13.00
TN per class: [54 60 57 41] Average: 53.00
FP per class: [ 6  0  3 19] Average: 7.00
FN per class: [ 7  6 11  4] Average: 7.00
ACCURACY per class: [0.8375 0.925  0.825  0.7125] Average: 0.83
PRECISION per class: [0.68421053 1.         0.75       0.45714286] Average: 0.72
RECALL per class: [0.65 0.7  0.45 0.8 ] Average: 0.65
F1 per class: [0.66666667 0.82352941 0.5625     0.58181818] Average: 0.66
Avg Loss val tensor(4.6792)
F1 val tensor(0.6586)
Accuracy VAL tensor(0.8250)


Validating: 0it [00:00, ?it/s]

Epoch 19, global step 6399: val_f1 was not in top 1



Eval stats:
TP per class: [19 14 20 14] Average: 16.75
TN per class: [57 60 51 59] Average: 56.75
FP per class: [3 0 9 1] Average: 3.25
FN per class: [1 6 0 6] Average: 3.25
ACCURACY per class: [0.95   0.925  0.8875 0.9125] Average: 0.92
PRECISION per class: [0.86363636 1.         0.68965517 0.93333333] Average: 0.87
RECALL per class: [0.95 0.7  1.   0.7 ] Average: 0.84
F1 per class: [0.9047619  0.82352941 0.81632653 0.8       ] Average: 0.84
Avg Loss val tensor(1.9574)
F1 val tensor(0.8362)
Accuracy VAL tensor(0.9219)
